<h1><div class="alert alert-block alert-info">
Genetic Algorithm
</div></h1>

<font face = "Times">The genetic algorithm is a method for solving both constrained and unconstrained optimization problems that is based on natural selection, the process that drives biological evolution. The genetic algorithm repeatedly modifies a population of individual solutions. At each step, the genetic algorithm selects individuals at random from the current population to be parents and uses them to produce the children for the next generation. Over successive generations, the population "evolves" toward an optimal solution. You can apply the genetic algorithm to solve a variety of optimization problems that are not well suited for standard optimization algorithms, including problems in which the objective function is discontinuous, nondifferentiable, stochastic, or highly nonlinear. The genetic algorithm can address problems of mixed integer programming, where some components are restricted to be integer-valued.

Source: https://fr.mathworks.com/help/gads/what-is-the-genetic-algorithm.html

The genetic algorithm iterates using three methods: selection, crossover and mutation, as presented in the digram below:

<img src="https://www.researchgate.net/profile/Peter_Steininger/publication/224123710/figure/fig1/AS:302573300011018@1449150504752/Principal-flow-of-a-Genetic-Algorithm-following-Goldberg-1989.png">

Source: https://www.researchgate.net/profile/Peter_Steininger/publication/224123710/figure/fig1/AS:302573300011018@1449150504752/Principal-flow-of-a-Genetic-Algorithm-following-Goldberg-1989.png 

The assessment of the quality of the population is measured through a fitness function, that must have the capacity to apply penalty to solutions that do not meet the constraints.</font>

In [1]:
#Used to install packages on the Python environment
import sys
!{sys.executable} -m pip install pyeasyga
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install random

Processing c:\users\fonse\appdata\local\pip\cache\wheels\ef\cf\ef\7aff9fcd6c1e59dc276182f29a32e7c197665dd5eb547f30e6\pyeasyga-0.3.1-py2.py3-none-any.whl
  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random


<h2><font color = "blue">Knapsack problem</font></h2>

<font face = "Times">The knapsack problem or rucksack problem is a problem in combinatorial optimization: Given a set of items, each with a weight and a value, determine the number of each item to include in a collection so that the total weight is less than or equal to a given limit and the total value is as large as possible. It derives its name from the problem faced by someone who is constrained by a fixed-size knapsack and must fill it with the most valuable items.

The problem often arises in resource allocation where there are financial constraints and is studied in fields such as combinatorics, computer science, complexity theory, cryptography and applied mathematics.

<img src = "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/Knapsack.svg/1024px-Knapsack.svg.png" height=30% width=30%>

<i>Source: Wikipedia</i></font>

In [456]:
from pyeasyga import pyeasyga
import pandas as pd
import random

#Tuple of the items that you can put on the knapsack, considering that you must maximise the 
#value and attend the constraint of maximum weight for the things you can carry on the knapsck
data = [{'name': 'item 1', 'value': 4, 'weight': 12},
        {'name': 'item 2', 'value': 2, 'weight': 1},
        {'name': 'item 3', 'value': 10, 'weight': 4},
        {'name': 'item 4', 'value': 1, 'weight': 1},
        {'name': 'item 5', 'value': 2, 'weight': 2}]


def genetic():
    ga = pyeasyga.GeneticAlgorithm(data,
                               population_size=100,
                               generations=20,
                               crossover_probability=0.15,
                               mutation_probability=0.1,
                               maximise_fitness=True)

    #Here I'm considering that the maximum quantity of each item that I want to test is 10 
    def create_individual(data):
        return [random.randint(0, 10) for _ in range(len(data))]

    ga.create_individual = create_individual

    # The fitness function indicates how we evaluate if a solution is good or not
    # for unfeasible solutions we penalize it giving a bad grade, in this case 0

    def fitness(individual, data):
        values, weights = 0, 0
        for selected, box in zip(individual, data):
            if selected:
                values += selected*box.get('value')
                weights += selected*box.get('weight')
        if weights > max_weight:
            values = 0
        return values

    #The goal is to maximize the value in the knapsack having as constraint the total weight 
    max_weight = 15

    ga.fitness_function = fitness               # set the GA's fitness function
    ga.run()                                    # run the GA

    df = pd.DataFrame()

    for i in range(0,len(data)):
        raw_data = {'Item': data[i]['name'], 
                    'Qty': ga.best_individual()[1][i],
                    'Weight': ga.best_individual()[1][i]*data[i]['weight'],
                    'Value': ga.best_individual()[1][i]*data[i]['value']}
        df = df.append(raw_data, ignore_index=True)

    df.style.hide_index()
    return df, ga

best_config = pd.DataFrame()
best_value = 0
for i in range(1,100):
    df, ga = genetic()
    if ga.best_individual()[0] > best_value:
        best_value = ga.best_individual()[0]
        best_config = df
        
best_config

,Item,Qty,Value,Weight
0,item 1,0.0,0.0,0.0
1,item 2,3.0,6.0,3.0
2,item 3,3.0,30.0,12.0
3,item 4,0.0,0.0,0.0
4,item 5,0.0,0.0,0.0


In [457]:
print("The best configuration has a total value of : ", best_value)
print("The best configuration has a total weight of: ", best_config['Weight'].sum())

The best configuration has a total value of :  36
The best configuration has a total weight of:  15.0
